In [12]:
!pip install transformers --user

You are using pip version 19.0.3, however version 22.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [13]:
!pip list

Package            Version   
------------------ ----------
certifi            2022.12.7 
charset-normalizer 3.0.1     
colorama           0.4.6     
filelock           3.9.0     
huggingface-hub    0.12.0    
idna               3.4       
importlib-metadata 6.0.0     
numpy              1.21.6    
packaging          23.0      
pip                19.0.3    
PyYAML             6.0       
regex              2022.10.31
requests           2.28.2    
setuptools         40.8.0    
tokenizers         0.13.2    
tqdm               4.64.1    
transformers       4.26.0    
typing-extensions  4.4.0     
urllib3            1.26.14   
zipp               3.12.0    


You are using pip version 19.0.3, however version 22.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import transformers

ModuleNotFoundError: No module named 'transformers'

In [ ]:
# create empty dataframe
temp = pd.DataFrame()
# combine datasets-
for i in range(0,6):
  # convert integer to string
  i = str(i)
  # get csv from internet
  df2 = pd.read_csv("data//" + i + ".csv")
  # add data to empty dataframe
  temp = pd.concat([temp,df2])
# set df to empty
df = temp
# encode method

df = df.reset_index()

In [ ]:
df.rename(columns={'lable':'Label'}, inplace=True)
df.drop(['index', 'id'], axis=1, inplace=True)
df = df.fillna('')

In [ ]:
df['Review'] = (df['method'].map(str) +' '+ df['user_agent'].map(str) +' '+ df['url'].map(str) +' '+ df['refer'].map(str) +' '+ df['body']).apply(lambda row: row.strip())
df

,method,user_agent,url,refer,body,Label,Review
0,GET,,/cv?uuid=69c0af3ce4caa8a131e86d0a879c55d6&tc=2...,,,0,GET /cv?uuid=69c0af3ce4caa8a131e86d0a879c55d6...
1,GET,,/110/20403/stodownload?m=d5ed3ca33799d2e0588af...,,,0,GET /110/20403/stodownload?m=d5ed3ca33799d2e0...
2,GET,Mozilla/5.0,/wpsv6internet/infos.ads??v=D1S1E2&d=arMFlS7Sp...,,,0,GET Mozilla/5.0 /wpsv6internet/infos.ads??v=D1...
3,GET,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,/dmfd/128_128_80/t01199cdfa0e3ec1bf2.jpg,,,0,GET Mozilla/5.0 (Windows NT 10.0; WOW64) Apple...
4,GET,Mozilla/4.0,/dl1.gj.qq.com/invc/mgrdataupdate/did52_2013.0...,,,0,GET Mozilla/4.0 /dl1.gj.qq.com/invc/mgrdataupd...
...,...,...,...,...,...,...,...
33032,GET,Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/2...,/webadmin/deny/index.php?dpid=1&dpruleid=1&cat...,,,5,GET Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gec...
33033,GET,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,/djvyr8op.pl?<script>cross_site_scripting.nasl...,,,5,GET Mozilla/4.0 (compatible; MSIE 8.0; Windows...
33034,GET,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,"/pan/loyalty_enu/start.swe/>"">/script><script>...",,,5,GET Mozilla/5.0 (Windows NT 6.3; WOW64) AppleW...
33035,GET,Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/2...,/wp-content/plugins/ultimate-weather-plugin/ma...,,,5,GET Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gec...


In [ ]:
data = df[['Review', 'Label']].copy()
data

,Review,Label
0,GET /cv?uuid=69c0af3ce4caa8a131e86d0a879c55d6...,0
1,GET /110/20403/stodownload?m=d5ed3ca33799d2e0...,0
2,GET Mozilla/5.0 /wpsv6internet/infos.ads??v=D1...,0
3,GET Mozilla/5.0 (Windows NT 10.0; WOW64) Apple...,0
4,GET Mozilla/4.0 /dl1.gj.qq.com/invc/mgrdataupd...,0
...,...,...
33032,GET Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gec...,5
33033,GET Mozilla/4.0 (compatible; MSIE 8.0; Windows...,5
33034,GET Mozilla/5.0 (Windows NT 6.3; WOW64) AppleW...,5
33035,GET Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gec...,5


In [ ]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Review'], data['Label'], test_size=0.2)

# Use the BERT tokenizer to convert the texts to sequences of tokens
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
train_sequences = tokenizer.batch_encode_plus(train_data, max_length=512, pad_to_max_length=True, return_attention_mask=True)
test_sequences = tokenizer.batch_encode_plus(test_data, max_length=512, pad_to_max_length=True, return_attention_mask=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Convert the labels to one-hot encoding
train_labels = pd.get_dummies(train_labels).values
test_labels = pd.get_dummies(test_labels).values

# Convert the sequences and labels to tensors
train_inputs = tf.convert_to_tensor(train_sequences['input_ids'])
train_attention_masks = tf.convert_to_tensor(train_sequences['attention_mask'])
train_labels = tf.convert_to_tensor(train_labels)
test_inputs = tf.convert_to_tensor(test_sequences['input_ids'])
test_attention_masks = tf.convert_to_tensor(test_sequences['attention_mask'])
test_labels = tf.convert_to_tensor(test_labels)

In [ ]:
# Define the input layers
input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='attention_mask')

# Use the BERT model
bert_model = transformers.TFBertModel.from_pretrained('bert-base-uncased')(input_ids, attention_mask)

# Extract the last hidden state from the BERT model
last_hidden_state = bert_model[0][:, 0, :]

# Add dropout and dense layers on top of the BERT model
x = tf.keras.layers.Dropout(0.3)(last_hidden_state)
x = tf.keras.layers.Dense(units=512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=len(train_labels[0]), activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# Train the model
history = model.fit(x=[train_inputs, train_attention_masks], y=train_labels, epochs=10, batch_size=32, validation_data=([test_inputs, test_attention_masks], test_labels))
history

Epoch 1/10


In [ ]:
# Evaluate the model on the test data
predictions = model.predict([test_inputs, test_attention_masks])
predictions = np.round(predictions)

# Calculate the confusion matrix
